In [1]:
%cd ../..
%load_ext autoreload

%autoreload 2

/home/giacomowsl/Projects/sinergia/emplusvault


In [2]:
import os
from pathlib import Path
import rts
from rts.utils import dataframe_from_hdf5
import rts.features.pose as pose
import numpy as np
import pandas as pd

/home/giacomowsl/.cache/pypoetry/virtualenvs/rts-o0uzL038-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
archiva_path = "/mnt/g/ioc/data/"
os.path.isdir(archiva_path)

True

In [6]:
data = dataframe_from_hdf5(archiva_path, "metadata")

In [7]:
data.head()

,guid,seq_id,date,duration,start,end,description,event,extra,duration_sec,sport,category,raw_round,details,round
0,MDA30021100,MDA30021100,2004-08-22,0 days 00:57:03,10:38:33.07,11:35:36.22,"Athens 2004 Summer OG, Cycling Track, Team pur...",Cycling Track/Men's Team Pursuit[M]/Qualifying,{},3423.0,Cycling,Men's Team Pursuit[M],Qualifying,,Preliminary
1,MDA30021100,SEQ41430642,2004-08-22,0 days 00:16:23,10:38:33.07,10:54:57.04,"Athens 2004 Summer OG, Cycling Track, Team pur...",Cycling Track/Men's Team Pursuit[M]/Qualifying,{},983.0,Cycling,Men's Team Pursuit[M],Qualifying,,Preliminary
2,MDA30021200,MDA30021200,2004-08-24,0 days 00:58:59,19:25:51.00,20:24:50.00,"Athens 2004 Summer OG, Synchronised swimming, ...",Artistic Swimming/Duet[W],{},3539.0,Swimming,Duet[W],,,
3,MDA30021200,SEQ41465934,2004-08-24,0 days 00:58:59,19:25:51.00,20:24:50.01,"Athens 2004 Summer OG, Synchronised swimming, ...",Artistic Swimming/Duet[W],{},3539.0,Swimming,Duet[W],,,
4,MDA30021300,MDA30021300,2004-08-25,0 days 00:59:30,22:36:00.00,23:35:30.01,"Athens 2004 Summer OG, Baseball, Final, Men, C...",Baseball/baseball(M)/Final,{},3570.0,Baseball,baseball(M),Final,,Final


In [8]:
sequences_folder = "/mnt/g/ioc/sequences/"
extracted_sequences = os.listdir(sequences_folder)
data = data[data.guid.map(lambda x: x in extracted_sequences)]
data = data[data.duration_sec < 600]

In [9]:
data.sport.value_counts()

sport
Basketball          11928
Volleyball          10144
Tennis               8571
Table Tennis         7905
Handball             6919
Baseball             6386
Beach Volleyball     6233
Canoeing             6061
Athletics            5810
Golf                 5676
Archery              5594
Taekwondo            5548
Judo                 4911
Weightlifting        4793
Badminton            4731
Fencing              4631
Boxing               4169
Cycling              4148
Equestrian           4030
Gymnastics           3871
Hockey               3193
Rugby                3094
Rowing               1901
Shooting             1673
Non-Sport             108
Name: count, dtype: int64

In [ ]:
top_sports = data.sport.value_counts().index.tolist()
top_sports.remove("Non-Sport")

In [12]:
data = data[data.sport.map(lambda x: x in top_sports)]

In [13]:
N_SEQS_PER_SPORT = 100
sample = data.groupby("sport").sample(N_SEQS_PER_SPORT)
len(sample)

2400

In [14]:
sequences_folder = "/mnt/g/ioc/sequences/"

sequences_videos = []
for root, dirs, files in os.walk(sequences_folder):
    for file in files:
        if file.endswith('.mp4'):
            sequences_videos.append(os.path.join(root, file))

print(len(sequences_videos))

608


In [15]:
data["video_path"] = data.guid.map(lambda x: [f for f in sequences_videos if x in f][0])

In [16]:
output_folder = "data/test_poses/"

data.apply(lambda df: pose.process_video(
    pose.PifPafModel.fast, 
    Path(output_folder + "%s.jl" %df["guid"]), 
    Path(df["video_path"]), 
    skip_frame=10
), axis = 1)

INFO:RTS:Processing /mnt/g/ioc/sequences/MDA43032700/SEQ43069526.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA43296000/SEQ43318192.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42723010/SEQ42724836.mp4


[mov,mp4,m4a,3gp,3g2,mj2 @ 0xa70fe40] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x8f28540] moov atom not found


INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42722830/SEQ42723011.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42755330/SEQ42755773.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42786350/SEQ42799523.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42716760/SEQ42721756.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42733160/SEQ42752722.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA43059660/SEQ43119458.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA43146460/SEQ43163620.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42746070/SEQ42763661.mp4


[mov,mp4,m4a,3gp,3g2,mj2 @ 0xb0a0240] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0xa811f40] moov atom not found


INFO:RTS:Processing /mnt/g/ioc/sequences/MDA43106780/SEQ43151663.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42970190/SEQ42975165.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42941911/SEQ42951815.mp4


...

INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42861619/SEQ42869166.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42762539/SEQ42784273.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42786339/SEQ42801795.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42786349/SEQ42800061.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42716759/SEQ42722597.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42740059/SEQ42760336.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42914559/SEQ42920768.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42918459/SEQ42939079.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA43059659/SEQ43119351.mp4


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x147c36c0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1ce85500] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1db81a80] moov atom not found


INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42936089/SEQ42954594.mp4
INFO:RTS:Processing /mnt/g/ioc/sequences/MDA42970189/SEQ42974853.mp4


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1ce85500] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x1ce85500] moov atom not found


11116      True
11117      True
11118      True
11119      True
11120      True
           ... 
2299991    True
2299992    True
2299993    True
2299994    True
2299995    True
Length: 131920, dtype: bool

In [17]:
poses_jl = [output_folder + f for f in os.listdir(output_folder)]
poses = []
for pose_json in poses_jl:
    poses.append(pose.extract_frame_data(pose_json, 0.5))

print(len(poses))

407
